In [ ]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from scipy import integrate
import AnaUtils as au
import Triggered as tr
import glob
import os
from pylab import rcParams
from matplotlib.offsetbox import AnchoredText
rcParams['figure.figsize'] = 15, 11
SMALL_SIZE = 22
MEDIUM_SIZE = 30
BIGGER_SIZE = 35
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
qe = 1.6022e-7

In [ ]:
pmt = "KA0196"
wdir = '/global/cfs/cdirs/lz/users/heuerman/UmichData'
#wdir = '/global/cfs/cdirs/lz/users/lkorley/UMPMT/OCS_1369V'
fWaves = [os.path.splitext(iF)[0] for iF in glob.glob('{}/Gain/{}*/*.bin'.format(wdir,pmt))]
#fWaves = [os.path.splitext(iF)[0] for iF in glob.glob('{}/*.bin'.format(wdir))]
fWaves.sort() 
Gains = np.array([float(iF.split('/')[-1].split('V')[0].split('_')[-1]) for iF in fWaves])
#Gains = np.array([float(iF.split('/')[-1].split('.')[0].split('_')[-1]) for iF in fWaves])
#Gains *= np.array([float(iF.split('/')[-1].split('.')[0].split('_')[-2]) for iF in fWaves])
nGains = len(Gains)
print(Gains,fWaves,nGains)

In [ ]:
QSPE = np.array([0.]*nGains)
QErr = np.array([0.]*nGains)
QSig = np.array([0.]*nGains)
QSigE = np.array([0.]*nGains)
ExSPE = {'V':0, 'Gain':0,'u':1e6, 'QSPE': None,'Fit':None,'FitPars':None}
for iq in range(nGains):
    #plt.plot(QHists[iq][1],QHists[iq][2],marker='x',label=Gains[iq])
    print("\n-----------------------------------------")
    QHists = tr.extRunCh(fWaves[iq],30,4,8,pmt=1,qbins=100,qW=0.05,plot=False,ret=True)
    args = QHists[0][3]
    lst = 'solid'
    plt.plot(QHists[0][1][args],QHists[0][0][args],marker='+',label=int(Gains[iq]),linestyle=lst)
    try:
        P = {'Q0':0,'Q1':1.6,'s0':0.2,'s1':0.8,'u':1.5}
        GFit,GErr = tr.fitQP(QHists[0],P,doErr=True)#,3.2,0.1,1])
        QSPE[iq] = GFit['Q1']
        QSig[iq] = GFit['s1']
        QSigE[iq] = np.sqrt(GErr['s1'])
        print("Bias = {}V".format(Gains[iq]))
        print("Fit Results: {}".format(GFit))
        print("Fit Errors (Square): {}".format(GErr))
        QErr[iq] = np.sqrt(GErr['Q1'])
        '''
        if abs(GFit['u']-0.1)<abs(ExSPE['u']-0.1):
            ExSPE['V'] = Gains[iq]
            ExSPE['u'] = GFit['u']
            ExSPE['Gain'] = QSPE[iq]/qe
            ExSPE['QSPE'] = (QHists[iq][0][1][args],QHists[iq][0][0][args])
            minQ = min(QHists[iq][0][1][args])
            maxQ = max(QHists[iq][0][1][args])
            xVals = np.arange(minQ,maxQ,1e-3)
            ExSPE['Fit'] = xVals,GFit['norm']*tr.gpn2(xVals,50,*list(GFit.values())[:-2])
            ExSPE['FitPars'] = {r'$Q_0$':ufloat(GFit['Q0'],GErr['Q0']),
                                r'$Q_1$':ufloat(GFit['Q1'],GErr['Q1']),
                                r'$\sigma_0$':ufloat(GFit['s0'],GErr['s0']),
                                r'$\sigma_1$':ufloat(GFit['s1'],GErr['s1']),
                                r'$\lambda$':ufloat(GFit['u'],GErr['u']),
                                r'$\chi^2$':GFit['chi2'][0]
                                }
        '''
    except RuntimeError as e:
        print("Failed to fit {}V Run: {}".format(Gains[iq],e))
        lst = 'dashed'
        QSPE[iq] = np.nan
        QErr[iq] = np.nan
        #plt.plot(QHists[iq][1],QHists[iq][0],marker='x',label=Gains[iq])
#plt.yscale('log')
plt.xlim(-2,10)
plt.yscale('log')
plt.legend()
plt.xlabel('Q [pC]')
plt.ylabel('dRate/dQ [Hz/pC]')
plt.savefig("{}/figs/{}_GainSPEs.png".format(wdir,pmt))

Gains,QSPE/qe,QErr/qe

In [ ]:
c0 = np.log(1e7)
def fit_gain_curve(v, v0, gamma):
    g = ( v / v0 )**gamma
    return g*1e7
def fit_gain_line(lnv,lnv0,gamma):
    g = gamma*(lnv - lnv0)
    return g+c0
from scipy.optimize import curve_fit
gainX = Gains
gainY = QSPE
gainErrs = np.log(QSPE/qe)*QErr/QSPE
print(gainX,gainY,gainErrs)
fitQ,fitErr = curve_fit(fit_gain_line,np.log(Gains),np.log(QSPE/qe),p0=[7,2],sigma=np.log(QSPE/qe)*QErr/QSPE,absolute_sigma=True)
fitErr = np.sqrt(np.diag(fitErr))
print(fitQ,fitErr)
fitErr[0] = np.exp(fitQ[0])*fitErr[0]/fitQ[0]
fitQ[0] = np.exp(fitQ[0])
print(fitQ,fitErr)
fitPar = unumpy.uarray(fitQ,fitErr)
print(fitPar)

"""
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(Gains,np.log(QSPE/qe))
print(slope, intercept, std_err,(np.log(1e7)-intercept)/slope)
"""


In [ ]:
rcParams['figure.figsize'] = 15, 11
SMALL_SIZE = 22
MEDIUM_SIZE = 30
BIGGER_SIZE = 35
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
minV = min(Gains)
maxV = max(Gains)
xV = np.arange(minV,maxV,1)
points = fit_gain_curve(xV,fitQ[0],fitQ[1])
#pointerr = np.sqrt(np.square(fitErr[1]/fitQ[1])+np.square(fitErr[0]/fitQ[0]))*points
plt.errorbar(Gains,QSPE/qe,label='data',yerr = QErr/qe)
plt.plot(xV,points,label='Fit',marker='.',linestyle='dashed')
#plt.yscale('log')
#plt.xlim(-2,10)
plt.xlabel('Bias [V]')
plt.ylabel('Gain')
plt.legend()
spar = [r'$\gamma$ = {:.1u}'.format(fitPar[1]),r'$V_{1E7}$'+' = {:.1u}'.format(fitPar[0])]
anctxt = AnchoredText('Fit Params:\n'+'\n'.join(np.array(spar)), loc=4)
plt.gca().add_artist(anctxt)
plt.savefig("{}/figs/{}_GainCurve.png".format(wdir,pmt))
plt.show()